# PrimoGPT NLP Features Evaluation

This notebook evaluates the predictive accuracy of NLP features generated by PrimoGPT model. The evaluation compares generated features against actual market movements in the following period.

## Evaluation Period
- Start Date: January 1, 2024
- End Date: July 31, 2024
- Purpose: Aligns with PrimoRL trading simulation phase

## Feature Evaluation
The notebook analyzes the predictive power of:
1. Combined evaluation algorithm (sentiment + trend)
2. Individual features:
   - News Relevance (0-2)
   - Sentiment (-1 to 1)
   - Price Impact Potential (-3 to 3)
   - Trend Direction (-1 to 1)
   - Earnings Impact (-2 to 2)
   - Investor Confidence (-3 to 3)
   - Risk Profile Change (-2 to 2)

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

In [2]:
csv_file_name = "trade_data/CRM_data.csv"

df = pd.read_csv(csv_file_name)

# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Filter data for the specified date range
start_date = '2024-01-01'
end_date = '2024-07-31'
df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

df.head()

,Date,Adj Close Price,Returns,Bin Label,News Relevance,Sentiment,Price Impact Potential,Trend Direction,Earnings Impact,Investor Confidence,Risk Profile Change,Prompt
585,2024-01-02,255.398315,-0.026640,D3,1,0,-1,-1,0,-1,-1,\n [COMPANY BASICS]\n ...
586,2024-01-03,251.120560,-0.016749,D2,1,0,-1,-1,0,-1,0,\n [COMPANY BASICS]\n ...
587,2024-01-04,250.522278,-0.002382,D1,2,1,2,1,1,2,0,\n [COMPANY BASICS]\n ...
588,2024-01-05,250.402603,-0.000478,D1,1,0,0,0,0,0,0,\n [COMPANY BASICS]\n ...
589,2024-01-08,260.124756,0.038826,U4,2,1,2,1,0,2,0,\n [COMPANY BASICS]\n ...


In [3]:
def predict_movement(row, previous_direction):
    if row['News Relevance'] == 0 or (row['Sentiment'] + row['Trend Direction'] == 0):
        return previous_direction
    
    score = row['Sentiment'] + row['Trend Direction']
    
    if score > 0:
        return 'U'
    else:
        return 'D'

def classify_actual_movement(bin_label):
    if pd.isna(bin_label) or bin_label is None:
        return 'U'  # Assume 'Up' for missing values (or you can choose 'D')
    bin_label = str(bin_label)  # Convert to string just in case
    if bin_label.startswith('U'):
        return 'U'
    else:
        return 'D'

# Add actual values for the current day
df['Actual_Current_Day'] = df['Bin Label'].apply(classify_actual_movement)

# Initialize a list for predictions
predictions = []

# Iterate through the DataFrame and predict for each row
previous_direction = 'U'  # Initial assumption for the first row
for _, row in df.iterrows():
    prediction = predict_movement(row, previous_direction)
    predictions.append(prediction)
    previous_direction = row['Actual_Current_Day']  # Update previous_direction for the next iteration

# Add predictions to the DataFrame
df['Predicted_Next_Day'] = predictions

# Shift actual values one day forward for comparison with predictions
df['Actual_Next_Day'] = df['Actual_Current_Day'].shift(-1)

# Remove the last row since we don't have an actual value for the next day after the last day
df = df.dropna(subset=['Actual_Next_Day'])

# Calculate accuracy
accuracy = accuracy_score(df['Actual_Next_Day'], df['Predicted_Next_Day'])
conf_matrix = confusion_matrix(df['Actual_Next_Day'], df['Predicted_Next_Day'], labels=['U', 'D'])

print(f"Točnost predviđanja: {accuracy:.2f}")
print("\nMatrica zabune:")
print(pd.DataFrame(conf_matrix, index=['Actual U', 'Actual D'], columns=['Pred U', 'Pred D']))

print("\nIzvještaj o klasifikaciji:")
print(classification_report(df['Actual_Next_Day'], df['Predicted_Next_Day']))

# Analyze the accuracy of the combination of sentiment and trend
correct_predictions = ((df['Sentiment'] + df['Trend Direction'] > 0) & (df['Actual_Next_Day'] == 'U')) | \
                      ((df['Sentiment'] + df['Trend Direction'] < 0) & (df['Actual_Next_Day'] == 'D')) | \
                      ((df['Sentiment'] + df['Trend Direction'] == 0) & (df['Predicted_Next_Day'] == df['Actual_Next_Day']))
combined_accuracy = correct_predictions.mean()
print(f"\nTočnost kombinacije Sentiment + Trend Direction: {combined_accuracy:.2f}")

Točnost predviđanja: 0.45

Matrica zabune:
          Pred U  Pred D
Actual U      41      35
Actual D      43      24

Izvještaj o klasifikaciji:
              precision    recall  f1-score   support

           D       0.41      0.36      0.38        67
           U       0.49      0.54      0.51        76

    accuracy                           0.45       143
   macro avg       0.45      0.45      0.45       143
weighted avg       0.45      0.45      0.45       143


Točnost kombinacije Sentiment + Trend Direction: 0.45


In [4]:
def analyze_feature_predictive_power(df, features):
    results = {}
    
    for feature in features:
        predictions = []
        previous_direction = 'U'  # Initial assumption for the first row
        
        for _, row in df.iterrows():
            if row[feature] > 0:
                prediction = 'U'
            elif row[feature] < 0:
                prediction = 'D'
            else:
                prediction = previous_direction
            
            predictions.append(prediction)
            previous_direction = row['Actual_Current_Day']
        
        accuracy = accuracy_score(df['Actual_Next_Day'], predictions)
        conf_matrix = confusion_matrix(df['Actual_Next_Day'], predictions, labels=['U', 'D'])
        report = classification_report(df['Actual_Next_Day'], predictions, output_dict=True)
        
        results[feature] = {
            'accuracy': accuracy,
            'confusion_matrix': conf_matrix,
            'classification_report': report
        }
    
    return results

# List of features we want to analyze
features_to_analyze = [
    "Sentiment", "Price Impact Potential", 
    "Trend Direction", "Earnings Impact", "Investor Confidence", "Risk Profile Change"
]

# We conduct the analysis
feature_analysis = analyze_feature_predictive_power(df, features_to_analyze)

# We print the results
for feature, results in feature_analysis.items():
    print(f"\nAnaliza za značajku: {feature}")
    print(f"Točnost: {results['accuracy']:.2f}")
    print("\nMatrica zabune:")
    print(pd.DataFrame(results['confusion_matrix'], index=['Actual U', 'Actual D'], columns=['Pred U', 'Pred D']))
    print("\nDetaljna analiza:")
    print(f"Precision (U): {results['classification_report']['U']['precision']:.2f}")
    print(f"Recall (U): {results['classification_report']['U']['recall']:.2f}")
    print(f"F1-score (U): {results['classification_report']['U']['f1-score']:.2f}")
    print(f"Precision (D): {results['classification_report']['D']['precision']:.2f}")
    print(f"Recall (D): {results['classification_report']['D']['recall']:.2f}")
    print(f"F1-score (D): {results['classification_report']['D']['f1-score']:.2f}")
    print("-" * 50)

# We sort features by accuracy
sorted_features = sorted(feature_analysis.items(), key=lambda x: x[1]['accuracy'], reverse=True)

print("\nZnačajke rangirane prema točnosti:")
for feature, results in sorted_features:
    print(f"{feature}: {results['accuracy']:.2f}")


Analiza za značajku: Sentiment
Točnost: 0.45

Matrica zabune:
          Pred U  Pred D
Actual U      44      32
Actual D      46      21

Detaljna analiza:
Precision (U): 0.49
Recall (U): 0.58
F1-score (U): 0.53
Precision (D): 0.40
Recall (D): 0.31
F1-score (D): 0.35
--------------------------------------------------

Analiza za značajku: Price Impact Potential
Točnost: 0.44

Matrica zabune:
          Pred U  Pred D
Actual U      36      40
Actual D      40      27

Detaljna analiza:
Precision (U): 0.47
Recall (U): 0.47
F1-score (U): 0.47
Precision (D): 0.40
Recall (D): 0.40
F1-score (D): 0.40
--------------------------------------------------

Analiza za značajku: Trend Direction
Točnost: 0.45

Matrica zabune:
          Pred U  Pred D
Actual U      41      35
Actual D      43      24

Detaljna analiza:
Precision (U): 0.49
Recall (U): 0.54
F1-score (U): 0.51
Precision (D): 0.41
Recall (D): 0.36
F1-score (D): 0.38
--------------------------------------------------

Analiza za značajku: